<a href="https://colab.research.google.com/github/leenago/Projects/blob/master/KDX/%ED%98%84%EA%B2%BD_OS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%config InlineBackend.figure_format = 'retina'
!apt -qq -y install fonts-nanum

The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 6 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Selecting previously unselected package fonts-nanum.
(Reading database ... 144617 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20170925-1_all.deb ...
Unpacking fonts-nanum (20170925-1) ...
Setting up fonts-nanum (20170925-1) ...
Processing triggers for fontconfig (2.12.6-0ubuntu2) ...


In [2]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm
import matplotlib.gridspec as gridspec
import numpy as np

fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager._rebuild()
mpl.pyplot.rc('font', family='NanumGothic')
fm._rebuild()

mpl.rcParams['axes.unicode_minus'] = False

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# 그룹별 소비업종 분석

In [4]:
mcorp = pd.read_csv('/content/gdrive/Shared drives/청년취업아카데미_KDX한국데이터거래소/Data/mcorp_edit.csv')
mcorp

,소비일자,소비업종,성별,연령대,OS유형,소비금액,소비건수
0,20190101,PC사무기기,F,10,IOS,352443,13
1,20190101,PC사무기기,F,10,WINDOWS,84000,1
2,20190101,PC사무기기,F,10,안드로이드,80870,8
3,20190101,PC사무기기,F,10,없음,3700,1
4,20190101,PC사무기기,F,20,IOS,27714776,381
...,...,...,...,...,...,...,...
1146150,20200630,휴대폰악세서리,M,70,WINDOWS,243920,8
1146151,20200630,휴대폰악세서리,M,70,안드로이드,241320,11
1146152,20200630,휴대폰악세서리,M,70,IOS,139800,1
1146153,20200630,휴대폰악세서리,M,70,WINDOWS,1392071,4


In [5]:
new_mcorp=mcorp.replace({'IOS':'Mobile','안드로이드':"Mobile",'WINDOWS':'PC'})
new_mcorp = new_mcorp[new_mcorp.OS유형!='없음']
new_mcorp

,소비일자,소비업종,성별,연령대,OS유형,소비금액,소비건수
0,20190101,PC사무기기,F,10,Mobile,352443,13
1,20190101,PC사무기기,F,10,PC,84000,1
2,20190101,PC사무기기,F,10,Mobile,80870,8
4,20190101,PC사무기기,F,20,Mobile,27714776,381
5,20190101,PC사무기기,F,20,PC,11414514,60
...,...,...,...,...,...,...,...
1146150,20200630,휴대폰악세서리,M,70,PC,243920,8
1146151,20200630,휴대폰악세서리,M,70,Mobile,241320,11
1146152,20200630,휴대폰악세서리,M,70,Mobile,139800,1
1146153,20200630,휴대폰악세서리,M,70,PC,1392071,4


In [68]:
def call_mcorp_category_data(year='2019', category='가전/가구'):
  new_mcorp['소비일자'] = new_mcorp['소비일자'].astype('str')
  df = pd.DataFrame()
  for month in ('01','02','03','04','05','06'):
    date=year+month
    new_df = new_mcorp[new_mcorp['소비일자'].str.contains(date)]
    df  = pd.concat([df,new_df])
  return df[df['소비업종']==category]

In [69]:
def rate(category='가전/가구'):
  df_2019 = pd.DataFrame(call_mcorp_category_data('2019',category).groupby(['연령대','OS유형'])['소비건수'].sum())
  df_2020 = pd.DataFrame(call_mcorp_category_data('2020',category).groupby(['연령대','OS유형'])['소비건수'].sum())
  df = pd.concat([df_2019,df_2020,df_2020-df_2019],axis=1)
  df.columns = ['2019년','2020년','차이']
  df['2019년'] = df['2019년'].astype('float64')
  df['2020년'] = df['2020년'].astype('float64')
  df['차이'] = df['차이'].astype('float64')
  #df['OS유형'] = df['OS유형'].astype('float64')
  df['증감률'] = df['차이'] / df['2019년'] * 100
  df = df.reset_index()
  return df

In [70]:
def rate_sex(category='가전/가구'):
  df_2019 = pd.DataFrame(call_mcorp_category_data('2019',category).groupby(['연령대','성별','OS유형'])['소비건수'].sum())
  df_2020 = pd.DataFrame(call_mcorp_category_data('2020',category).groupby(['연령대','성별','OS유형'])['소비건수'].sum())
  df = pd.concat([df_2019,df_2020,df_2020-df_2019],axis=1)
  df.columns = ['2019년','2020년','차이']
  df['2019년'] = df['2019년'].astype('float64')
  df['2020년'] = df['2020년'].astype('float64')
  df['차이'] = df['차이'].astype('float64')
  #df['OS'] = df['OS유형'].astype('float64')
  df['증감률'] = df['차이'] / df['2019년'] * 100
  df = df.reset_index()
  return df

## GROUP 01. 코로나 관련 용품
- 건강관련용품
- 안전용품
- 업소위생용품

In [72]:
rate(category='건강관련용품')

,연령대,OS유형,2019년,2020년,차이,증감률
0,10,Mobile,7980.0,14563.0,6583.0,82.493734
1,10,PC,524.0,416.0,-108.0,-20.610687
2,20,Mobile,162539.0,489397.0,326858.0,201.095122
3,20,PC,11709.0,17864.0,6155.0,52.566402
4,30,Mobile,325211.0,1259688.0,934477.0,287.344832
5,30,PC,43032.0,82461.0,39429.0,91.627161
6,40,Mobile,224962.0,1050829.0,825867.0,367.114001
7,40,PC,65701.0,160737.0,95036.0,144.649244
8,50,Mobile,75711.0,283560.0,207849.0,274.529461
9,50,PC,46169.0,121527.0,75358.0,163.222075


In [11]:
건강관련용품 = rate_sex(category='건강관련용품')
건강관련용품

,연령대,성별,OS유형,2019년,2020년,차이,증감률
0,10,F,Mobile,4955.0,10395.0,5440.0,109.788093
1,10,F,PC,194.0,185.0,-9.0,-4.639175
2,10,M,Mobile,3025.0,4168.0,1143.0,37.785124
3,10,M,PC,330.0,231.0,-99.0,-30.000000
4,20,F,Mobile,123917.0,414884.0,290967.0,234.807976
5,20,F,PC,6119.0,11235.0,5116.0,83.608433
6,20,M,Mobile,38622.0,74513.0,35891.0,92.928901
7,20,M,PC,5590.0,6629.0,1039.0,18.586762
8,30,F,Mobile,245769.0,1019854.0,774085.0,314.964458
9,30,F,PC,19573.0,42059.0,22486.0,114.882747


In [12]:
rate(category='안전용품').sort_values('증감률')

,연령대,OS유형,2019년,2020년,차이,증감률
1,10,PC,152.0,178.0,26.0,17.105263
3,20,PC,4489.0,5318.0,829.0,18.467365
5,30,PC,16838.0,26061.0,9223.0,54.774914
7,40,PC,29679.0,46494.0,16815.0,56.656222
9,50,PC,21378.0,37209.0,15831.0,74.052765
0,10,Mobile,1516.0,2790.0,1274.0,84.036939
11,60,PC,8495.0,15895.0,7400.0,87.110065
13,70,PC,1541.0,3236.0,1695.0,109.993511
2,20,Mobile,33514.0,86562.0,53048.0,158.286089
12,70,Mobile,570.0,1565.0,995.0,174.561404


In [13]:
안전용품 = rate_sex(category='안전용품')
안전용품.sort_values('증감률')

,연령대,성별,OS유형,2019년,2020년,차이,증감률
7,20,M,PC,2202.0,2403.0,201.0,9.128065
1,10,F,PC,54.0,59.0,5.0,9.259259
3,10,M,PC,98.0,119.0,21.0,21.428571
5,20,F,PC,2287.0,2915.0,628.0,27.459554
11,30,M,PC,9865.0,15163.0,5298.0,53.705018
13,40,F,PC,11579.0,17857.0,6278.0,54.218844
9,30,F,PC,6973.0,10898.0,3925.0,56.288542
15,40,M,PC,18100.0,28637.0,10537.0,58.215470
19,50,M,PC,14643.0,25114.0,10471.0,71.508571
2,10,M,Mobile,578.0,1008.0,430.0,74.394464


In [14]:
rate(category='업소위생용품').sort_values('증감률')

,연령대,OS유형,2019년,2020년,차이,증감률
1,10,PC,14.0,14.0,0.0,0.000000
3,20,PC,704.0,953.0,249.0,35.369318
0,10,Mobile,226.0,333.0,107.0,47.345133
5,30,PC,2978.0,4558.0,1580.0,53.055742
7,40,PC,4955.0,8737.0,3782.0,76.326942
9,50,PC,3640.0,7534.0,3894.0,106.978022
11,60,PC,1582.0,3290.0,1708.0,107.964602
2,20,Mobile,5065.0,10787.0,5722.0,112.971372
4,30,Mobile,12275.0,29623.0,17348.0,141.327902
12,70,Mobile,126.0,336.0,210.0,166.666667


In [15]:
업소위생용품 = rate_sex(category='업소위생용품')
업소위생용품.sort_values('증감률')

,연령대,성별,OS유형,2019년,2020년,차이,증감률
3,10,M,PC,12.0,7.0,-5.0,-41.666667
2,10,M,Mobile,110.0,83.0,-27.0,-24.545455
7,20,M,PC,360.0,425.0,65.0,18.055556
6,20,M,Mobile,1623.0,2322.0,699.0,43.068392
11,30,M,PC,1778.0,2634.0,856.0,48.143982
5,20,F,PC,344.0,528.0,184.0,53.488372
9,30,F,PC,1200.0,1924.0,724.0,60.333333
15,40,M,PC,3214.0,5293.0,2079.0,64.685750
10,30,M,Mobile,4126.0,7057.0,2931.0,71.037324
13,40,F,PC,1741.0,3444.0,1703.0,97.817346


## GROUP 02. 식료품
- 가공식품
- 농축수산물
- 건강식품

In [16]:
print(rate(category='가공식품')[2:6])
print(rate(category='농축수산물')[2:6])
print(rate(category='건강식품')[2:6])


   연령대    OS유형      2019년      2020년         차이        증감률
2   20  Mobile  4096671.0  6066288.0  1969617.0  48.078476
3   20      PC    63088.0    49813.0   -13275.0 -21.042037
4   30  Mobile  5272218.0  8289537.0  3017319.0  57.230543
5   30      PC   202764.0   187779.0   -14985.0  -7.390365
   연령대    OS유형      2019년      2020년         차이        증감률
2   20  Mobile  1014740.0  1502445.0   487705.0  48.062065
3   20      PC    33766.0    26318.0    -7448.0 -22.057691
4   30  Mobile  1957374.0  3050440.0  1093066.0  55.843492
5   30      PC   123597.0   118811.0    -4786.0  -3.872262
   연령대    OS유형     2019년      2020년        차이        증감률
2   20  Mobile  411446.0   485554.0   74108.0  18.011598
3   20      PC   29439.0    20024.0   -9415.0 -31.981385
4   30  Mobile  890222.0  1129427.0  239205.0  26.870264
5   30      PC  101795.0    87596.0  -14199.0 -13.948622


In [17]:
rate(category='가공식품')[2:7]

,연령대,OS유형,2019년,2020년,차이,증감률
2,20,Mobile,4096671.0,6066288.0,1969617.0,48.078476
3,20,PC,63088.0,49813.0,-13275.0,-21.042037
4,30,Mobile,5272218.0,8289537.0,3017319.0,57.230543
5,30,PC,202764.0,187779.0,-14985.0,-7.390365
6,40,Mobile,2651871.0,5217950.0,2566079.0,96.764850


In [18]:
가공식품 = rate_sex(category='가공식품')
가공식품.sort_values('증감률')

,연령대,성별,OS유형,2019년,2020년,차이,증감률
1,10,F,PC,1228.0,620.0,-608.0,-49.511401
3,10,M,PC,1301.0,850.0,-451.0,-34.665642
7,20,M,PC,29927.0,23412.0,-6515.0,-21.769639
5,20,F,PC,33161.0,26401.0,-6760.0,-20.385392
9,30,F,PC,96187.0,83099.0,-13088.0,-13.606828
11,30,M,PC,106577.0,104680.0,-1897.0,-1.779934
13,40,F,PC,148635.0,158996.0,10361.0,6.970767
2,10,M,Mobile,94006.0,113541.0,19535.0,20.780588
15,40,M,PC,150542.0,182695.0,32153.0,21.358159
17,50,F,PC,95336.0,118183.0,22847.0,23.964714


In [19]:
rate(category='농축수산물').sort_values('증감률')

,연령대,OS유형,2019년,2020년,차이,증감률
3,20,PC,33766.0,26318.0,-7448.0,-22.057691
5,30,PC,123597.0,118811.0,-4786.0,-3.872262
1,10,PC,846.0,831.0,-15.0,-1.773050
7,40,PC,234713.0,270758.0,36045.0,15.357053
9,50,PC,185109.0,239749.0,54640.0,29.517744
0,10,Mobile,34204.0,48236.0,14032.0,41.024442
11,60,PC,72356.0,105072.0,32716.0,45.215324
2,20,Mobile,1014740.0,1502445.0,487705.0,48.062065
13,70,PC,19020.0,28718.0,9698.0,50.988433
4,30,Mobile,1957374.0,3050440.0,1093066.0,55.843492


In [20]:
농축수산물 = rate_sex(category='농축수산물')
농축수산물.sort_values('증감률')

,연령대,성별,OS유형,2019년,2020년,차이,증감률
3,10,M,PC,421.0,314.0,-107.0,-25.415677
7,20,M,PC,16525.0,12477.0,-4048.0,-24.496218
5,20,F,PC,17241.0,13841.0,-3400.0,-19.720434
9,30,F,PC,61632.0,54920.0,-6712.0,-10.890447
11,30,M,PC,61965.0,63891.0,1926.0,3.108206
13,40,F,PC,123657.0,135571.0,11914.0,9.634715
2,10,M,Mobile,12571.0,14767.0,2196.0,17.468777
1,10,F,PC,425.0,517.0,92.0,21.647059
15,40,M,PC,111056.0,135187.0,24131.0,21.728677
17,50,F,PC,95214.0,120476.0,25262.0,26.531813


In [21]:
rate(category='건강식품').sort_values('증감률')

,연령대,OS유형,2019년,2020년,차이,증감률
1,10,PC,799.0,498.0,-301.0,-37.672090
3,20,PC,29439.0,20024.0,-9415.0,-31.981385
5,30,PC,101795.0,87596.0,-14199.0,-13.948622
7,40,PC,156993.0,156733.0,-260.0,-0.165612
9,50,PC,116995.0,137601.0,20606.0,17.612718
2,20,Mobile,411446.0,485554.0,74108.0,18.011598
0,10,Mobile,12124.0,15282.0,3158.0,26.047509
4,30,Mobile,890222.0,1129427.0,239205.0,26.870264
11,60,PC,44457.0,59489.0,15032.0,33.812448
6,40,Mobile,714307.0,1029929.0,315622.0,44.185763


In [22]:
건강식품 = rate_sex(category='건강식품')
건강식품.sort_values('증감률')

,연령대,성별,OS유형,2019년,2020년,차이,증감률
1,10,F,PC,297.0,175.0,-122.0,-41.077441
3,10,M,PC,502.0,323.0,-179.0,-35.657371
5,20,F,PC,15262.0,10280.0,-4982.0,-32.643166
7,20,M,PC,14177.0,9744.0,-4433.0,-31.268957
9,30,F,PC,50165.0,39913.0,-10252.0,-20.436559
11,30,M,PC,51630.0,47683.0,-3947.0,-7.644780
13,40,F,PC,77326.0,73922.0,-3404.0,-4.402142
15,40,M,PC,79667.0,82811.0,3144.0,3.946427
2,10,M,Mobile,4590.0,4788.0,198.0,4.313725
6,20,M,Mobile,100585.0,108436.0,7851.0,7.805339


## GROUP 03. 생필품
- 헤어바디용품
- 생활용품
- 세탁청소세면

In [23]:
print(rate(category='헤어바디용품')[2:6])
print(rate(category='생활용품')[2:6])
print(rate(category='세탁청소세면')[2:6])

   연령대    OS유형     2019년     2020년        차이        증감률
2   20  Mobile  339636.0  448469.0  108833.0  32.044012
3   20      PC   20053.0   15285.0   -4768.0 -23.776991
4   30  Mobile  573512.0  816678.0  243166.0  42.399462
5   30      PC   59086.0   59300.0     214.0   0.362184
   연령대    OS유형     2019년     2020년       차이        증감률
2   20  Mobile  225168.0  250500.0  25332.0  11.250266
3   20      PC   15862.0   11729.0  -4133.0 -26.055983
4   30  Mobile  714591.0  783591.0  69000.0   9.655873
5   30      PC   81336.0   70461.0 -10875.0 -13.370463
   연령대    OS유형     2019년     2020년       차이        증감률
2   20  Mobile  298247.0  323343.0  25096.0   8.414502
3   20      PC   20911.0   13848.0  -7063.0 -33.776481
4   30  Mobile  804409.0  888893.0  84484.0  10.502617
5   30      PC   89049.0   69597.0 -19452.0 -21.844153


In [24]:
rate(category='헤어바디용품').sort_values('증감률')


,연령대,OS유형,2019년,2020년,차이,증감률
1,10,PC,722.0,487.0,-235.0,-32.548476
3,20,PC,20053.0,15285.0,-4768.0,-23.776991
5,30,PC,59086.0,59300.0,214.0,0.362184
7,40,PC,83572.0,102419.0,18847.0,22.551812
2,20,Mobile,339636.0,448469.0,108833.0,32.044012
9,50,PC,55427.0,76422.0,20995.0,37.878651
4,30,Mobile,573512.0,816678.0,243166.0,42.399462
0,10,Mobile,14121.0,20545.0,6424.0,45.492529
8,50,Mobile,152864.0,237330.0,84466.0,55.255652
6,40,Mobile,425389.0,666969.0,241580.0,56.790373


In [25]:
헤어바디용품 = rate_sex(category='헤어바디용품')
헤어바디용품.sort_values('증감률')
# 노년층 남성이 급증

,연령대,성별,OS유형,2019년,2020년,차이,증감률
3,10,M,PC,349.0,202.0,-147.0,-42.120344
7,20,M,PC,8928.0,6111.0,-2817.0,-31.552419
1,10,F,PC,373.0,285.0,-88.0,-23.592493
5,20,F,PC,11125.0,9174.0,-1951.0,-17.537079
9,30,F,PC,31867.0,30340.0,-1527.0,-4.791791
11,30,M,PC,27219.0,28960.0,1741.0,6.396267
13,40,F,PC,46136.0,51144.0,5008.0,10.854864
6,20,M,Mobile,58373.0,71118.0,12745.0,21.833724
17,50,F,PC,29972.0,36748.0,6776.0,22.607767
2,10,M,Mobile,3434.0,4282.0,848.0,24.694234


In [26]:
rate(category='생활용품').sort_values('증감률')


,연령대,OS유형,2019년,2020년,차이,증감률
3,20,PC,15862.0,11729.0,-4133.0,-26.055983
1,10,PC,1213.0,1021.0,-192.0,-15.828524
5,30,PC,81336.0,70461.0,-10875.0,-13.370463
0,10,Mobile,6719.0,6812.0,93.0,1.384135
7,40,PC,119825.0,122595.0,2770.0,2.311705
4,30,Mobile,714591.0,783591.0,69000.0,9.655873
2,20,Mobile,225168.0,250500.0,25332.0,11.250266
9,50,PC,71701.0,87229.0,15528.0,21.656602
6,40,Mobile,479888.0,613327.0,133439.0,27.806280
8,50,Mobile,132549.0,173246.0,40697.0,30.703363


In [27]:
생활용품 = rate_sex(category='생활용품')
생활용품.sort_values('증감률')
# 노년층 남성이 40% 이상 증가

,연령대,성별,OS유형,2019년,2020년,차이,증감률
7,20,M,PC,6276.0,4585.0,-1691.0,-26.943913
5,20,F,PC,9586.0,7144.0,-2442.0,-25.474651
2,10,M,Mobile,2100.0,1568.0,-532.0,-25.333333
3,10,M,PC,184.0,138.0,-46.0,-25.000000
11,30,M,PC,42646.0,35820.0,-6826.0,-16.006190
1,10,F,PC,1029.0,883.0,-146.0,-14.188533
9,30,F,PC,38690.0,34641.0,-4049.0,-10.465236
13,40,F,PC,56630.0,53288.0,-3342.0,-5.901466
6,20,M,Mobile,34826.0,34370.0,-456.0,-1.309367
10,30,M,Mobile,144733.0,153197.0,8464.0,5.848010


In [28]:
rate(category='세탁청소세면').sort_values('증감률')

,연령대,OS유형,2019년,2020년,차이,증감률
1,10,PC,526.0,308.0,-218.0,-41.444867
3,20,PC,20911.0,13848.0,-7063.0,-33.776481
5,30,PC,89049.0,69597.0,-19452.0,-21.844153
7,40,PC,133278.0,128830.0,-4448.0,-3.337385
0,10,Mobile,8108.0,8431.0,323.0,3.983720
2,20,Mobile,298247.0,323343.0,25096.0,8.414502
4,30,Mobile,804409.0,888893.0,84484.0,10.502617
9,50,PC,88586.0,100253.0,11667.0,13.170253
6,40,Mobile,601432.0,743891.0,142459.0,23.686635
11,60,PC,31646.0,40523.0,8877.0,28.050939


In [29]:
세탁청소세면 = rate_sex(category='세탁청소세면')
세탁청소세면.sort_values('증감률')

## GROUP 04. 취미 용품
- 완구키덜트용품
- 주방수납잡화
-  헬스기구용품

In [30]:
print(rate(category='완구키덜트게임')[10:14])
print(rate(category='주방수납잡화')[10:14])
print(rate(category='헬스기구용품')[10:14])

    연령대    OS유형    2019년    2020년      차이        증감률
10   60  Mobile   9938.0  18447.0  8509.0  85.620849
11   60      PC  14358.0  16313.0  1955.0  13.616103
12   70  Mobile   1640.0   3247.0  1607.0  97.987805
13   70      PC   3042.0   3194.0   152.0   4.996713
    연령대    OS유형    2019년    2020년      차이        증감률
10   60  Mobile  12936.0  19839.0  6903.0  53.362709
11   60      PC  14450.0  19816.0  5366.0  37.134948
12   70  Mobile   2120.0   3142.0  1022.0  48.207547
13   70      PC   2871.0   4076.0  1205.0  41.971439
    연령대    OS유형   2019년   2020년     차이        증감률
10   60  Mobile  2046.0  2492.0  446.0  21.798631
11   60      PC  1859.0  2411.0  552.0  29.693384
12   70  Mobile   291.0   419.0  128.0  43.986254
13   70      PC   440.0   525.0   85.0  19.318182


In [31]:
rate(category='완구키덜트게임').sort_values('증감률')

,연령대,OS유형,2019년,2020년,차이,증감률
5,30,PC,90294.0,70756.0,-19538.0,-21.638204
7,40,PC,111907.0,101140.0,-10767.0,-9.621382
3,20,PC,25378.0,24546.0,-832.0,-3.278430
13,70,PC,3042.0,3194.0,152.0,4.996713
9,50,PC,41281.0,44418.0,3137.0,7.599138
1,10,PC,2105.0,2300.0,195.0,9.263658
11,60,PC,14358.0,16313.0,1955.0,13.616103
4,30,Mobile,613244.0,761009.0,147765.0,24.095629
2,20,Mobile,220561.0,284633.0,64072.0,29.049560
0,10,Mobile,17948.0,23856.0,5908.0,32.917317


In [32]:
완구키덜트게임 = rate_sex(category='완구키덜트게임')
완구키덜트게임.sort_values('증감률')

,연령대,성별,OS유형,2019년,2020년,차이,증감률
9,30,F,PC,34737.0,25159.0,-9578.0,-27.572905
11,30,M,PC,55557.0,45597.0,-9960.0,-17.927534
25,70,F,PC,1360.0,1203.0,-157.0,-11.544118
13,40,F,PC,42826.0,38508.0,-4318.0,-10.082660
15,40,M,PC,69081.0,62632.0,-6449.0,-9.335418
7,20,M,PC,16536.0,15555.0,-981.0,-5.932511
17,50,F,PC,16660.0,16638.0,-22.0,-0.132053
5,20,F,PC,8842.0,8991.0,149.0,1.685139
1,10,F,PC,536.0,564.0,28.0,5.223881
23,60,M,PC,9031.0,9942.0,911.0,10.087476


In [33]:
rate(category='주방수납잡화').sort_values('증감률')

,연령대,OS유형,2019년,2020년,차이,증감률
1,10,PC,143.0,106.0,-37.0,-25.874126
3,20,PC,8596.0,7656.0,-940.0,-10.935319
5,30,PC,39618.0,36829.0,-2789.0,-7.039729
7,40,PC,57844.0,62761.0,4917.0,8.500449
9,50,PC,38197.0,47457.0,9260.0,24.242742
4,30,Mobile,264024.0,347235.0,83211.0,31.516453
2,20,Mobile,88696.0,118235.0,29539.0,33.303644
11,60,PC,14450.0,19816.0,5366.0,37.134948
0,10,Mobile,2858.0,3961.0,1103.0,38.593422
13,70,PC,2871.0,4076.0,1205.0,41.971439


In [34]:
주방수납잡화 = rate_sex(category='주방수납잡화')
주방수납잡화

,연령대,성별,OS유형,2019년,2020년,차이,증감률
0,10,F,Mobile,1895.0,2804.0,909.0,47.968338
1,10,F,PC,85.0,74.0,-11.0,-12.941176
2,10,M,Mobile,963.0,1157.0,194.0,20.145379
3,10,M,PC,58.0,32.0,-26.0,-44.827586
4,20,F,Mobile,68820.0,91837.0,23017.0,33.445219
5,20,F,PC,5198.0,4604.0,-594.0,-11.427472
6,20,M,Mobile,19876.0,26398.0,6522.0,32.813443
7,20,M,PC,3398.0,3052.0,-346.0,-10.182460
8,30,F,Mobile,205496.0,267807.0,62311.0,30.322245
9,30,F,PC,19054.0,15952.0,-3102.0,-16.280046


In [35]:
rate(category='헬스기구용품')

,연령대,OS유형,2019년,2020년,차이,증감률
0,10,Mobile,2691.0,3234.0,543.0,20.178372
1,10,PC,321.0,199.0,-122.0,-38.006231
2,20,Mobile,30001.0,40409.0,10408.0,34.692177
3,20,PC,3296.0,2493.0,-803.0,-24.362864
4,30,Mobile,36713.0,45968.0,9255.0,25.209054
5,30,PC,5549.0,4713.0,-836.0,-15.065778
6,40,Mobile,27702.0,34409.0,6707.0,24.211248
7,40,PC,8171.0,7841.0,-330.0,-4.038673
8,50,Mobile,10636.0,12470.0,1834.0,17.243325
9,50,PC,6070.0,6953.0,883.0,14.546952


In [36]:
헬스기구용품 = rate_sex(category='헬스기구용품')
헬스기구용품.sort_values('증감률')

,연령대,성별,OS유형,2019년,2020년,차이,증감률
1,10,F,PC,106.0,41.0,-65.0,-61.320755
3,10,M,PC,215.0,158.0,-57.0,-26.511628
7,20,M,PC,2497.0,1874.0,-623.0,-24.949940
5,20,F,PC,799.0,619.0,-180.0,-22.528160
11,30,M,PC,3843.0,3244.0,-599.0,-15.586781
9,30,F,PC,1706.0,1469.0,-237.0,-13.892145
13,40,F,PC,3296.0,2887.0,-409.0,-12.408981
25,70,F,PC,163.0,162.0,-1.0,-0.613497
15,40,M,PC,4875.0,4954.0,79.0,1.620513
16,50,F,Mobile,6758.0,7419.0,661.0,9.781000
